## Homework 3
### Paweł Fijałkowski
#### XAI WB 2022L

In [31]:
import pandas as pd
import pickle as pkl
import numpy as np
import dalex as dx
from math import pi
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

#### Data fetch and feature engineering

In [4]:
data = pd.read_csv("EPL_2021.csv")

In [5]:
data['result'] = (data['result'] == 'Goal')
data['distance'] = ((105 - (data['X'] * 105)) ** 2 + (32.5 - (data['Y'] * 68)) ** 2) ** 0.5
data["angle"] = np.abs(np.arctan((7.32 * (105 - (data['X'] * 105))) / ((105 - (data['X'] * 105))**2 + (32.5 - (data['Y'] * 68)) ** 2 - (7.32 / 2) ** 2)) * 180 / pi)
data = data[['result', 'h_a', 'situation', 'shotType', 'lastAction', 'minute', 'distance', 'angle']]

In [6]:
data.head()

,result,h_a,situation,shotType,lastAction,minute,distance,angle
0,False,h,OpenPlay,Head,Aerial,10,10.034305,37.453252
1,False,h,OpenPlay,RightFoot,Throughball,11,14.699726,19.232346
2,True,h,OpenPlay,RightFoot,BallRecovery,21,19.973838,14.099715
3,False,h,OpenPlay,RightFoot,Pass,27,19.740004,21.007894
4,False,h,OpenPlay,RightFoot,Chipped,29,14.008206,24.418589


In [27]:
categorical_features = (data.dtypes == object)

In [28]:
data.loc[:,categorical_features] = data.loc[:,categorical_features].apply(LabelEncoder().fit_transform)

In [30]:
data.head()

,result,h_a,situation,shotType,lastAction,minute,distance,angle
0,False,1,2,0,0,10,10.034305,37.453252
1,False,1,2,3,27,11,14.699726,19.232346
2,True,1,2,3,1,21,19.973838,14.099715
3,False,1,2,3,20,27,19.740004,21.007894
4,False,1,2,3,6,29,14.008206,24.418589


#### Models & predictions

In [32]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('result', axis=1), data['result'], test_size=0.15)

In [33]:
logistic_regression, random_forest, xgboost = LogisticRegression(), RandomForestClassifier(), GradientBoostingClassifier()
models = [logistic_regression, random_forest, xgboost] 

In [34]:
for model in models:
    model.fit(X_train, y_train)


In [35]:
for model in models:
    print(f"Score for {f'{model=}'.split('=')[0]}: {model.score(X_test, y_test)}")

Score: 0.8791704238052299
Score: 0.8854824165915239
Score: 0.8863841298467088
